In [1]:
!pip install pyspark


In [2]:
from google.colab import files
uploaded = files.upload()


Saving friends.txt to friends.txt


In [8]:
# Import PySpark
from pyspark.sql import SparkSession

# Créer une session Spark
spark = SparkSession.builder.appName("AmisCommuns").getOrCreate()

# Charger les données en texte brut
data = spark.read.text("/content/friends.txt")


# Extraire la ligne brute
lines = data.rdd.map(lambda r: r[0])

# Afficher quelques lignes pour vérifier
print(lines.take(5))


['1 Sidi 2,3,4', '2 Mohamed 1,3,5', '3 Aicha 1,2,4,6', '4 Ahmed 1,3', '5 Leila 2']


In [9]:
def pairs(line):
    parts = line.split(" ")
    if len(parts) < 3:
        return []
    user = parts[0]
    friends = parts[2].split(",")
    result = []
    for friend in friends:
        # Trier les utilisateurs pour éviter les doublons (1,2) au lieu de (2,1)
        pair = tuple(sorted([user, friend]))
        result.append((pair, set(friends)))
    return result

# Appliquer la fonction
pairs_rdd = lines.flatMap(pairs)

print(pairs_rdd.take(10))


[(('1', '2'), {'4', '3', '2'}), (('1', '3'), {'4', '3', '2'}), (('1', '4'), {'4', '3', '2'}), (('1', '2'), {'1', '5', '3'}), (('2', '3'), {'1', '5', '3'}), (('2', '5'), {'1', '5', '3'}), (('1', '3'), {'1', '4', '6', '2'}), (('2', '3'), {'1', '4', '6', '2'}), (('3', '4'), {'1', '4', '6', '2'}), (('3', '6'), {'1', '4', '6', '2'})]


In [10]:
# Réduire par clé en intersection des sets
mutual_friends_rdd = pairs_rdd.reduceByKey(lambda s1, s2: s1.intersection(s2))

print(mutual_friends_rdd.take(10))


[(('1', '2'), {'3'}), (('1', '3'), {'4', '2'}), (('1', '4'), {'3'}), (('2', '3'), {'1'}), (('2', '5'), set()), (('3', '4'), {'1'}), (('3', '6'), set())]


In [11]:
# Définir la paire normalisée
target_pair = ('1', '2')

# Filtrer pour la paire ciblée
result = mutual_friends_rdd.filter(lambda x: x[0] == target_pair).collect()

# Affichage formaté
if result:
    amis_communs = ",".join(result[0][1])
    print(f"{target_pair[0]}<Sidi>{target_pair[1]}<Mohamed>{amis_communs}")
else:
    print("Aucun ami commun trouvé entre Sidi et Mohamed.")


1<Sidi>2<Mohamed>3
